In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
names = ['busy', 'threads', 'runtime', 'usertime', 'energy']
usecols = [0, 1, 3, 4, 5]
df = pd.read_csv('data/matmul.csv', names=names, usecols=usecols)
df

,busy,threads,runtime,usertime,energy
0,0,1,0.002060,0.024597,2.059994e+06
1,0,2,6.190170,117.389714,6.190170e+09
2,0,3,0.000464,0.000000,4.644960e+05
3,0,4,0.000000,0.000404,5.889490e-04
4,0,5,0.000000,0.000392,6.528290e-04
...,...,...,...,...,...
59,12,12,1.322080,0.037919,1.736086e-01
60,12,13,0.059964,2.108087,5.996438e+07
61,12,14,0.039995,1.394589,3.999486e+07
62,12,15,3.195731,0.090468,5.322391e-01
